In [1]:
"""
2021-08-30
가사를 저장하는 리스트가 2개인 것은 본인이 추후 어떻게 쓰일지 몰라 당장은 비합리적으로 보일수 있게 설계한 
"가사를 저장하는 방식"으로 인해 존재하는 코드입니다.
현재 가사를 검색한 사이트 별로 가사를 따로 저장하고 있기에, 
이러한 저장방식을 폐기하고 하나로 통합한다면
코드를 적지 않게 압축할 수 있을 것으로 보고 있습니다.

2021-08-31
현재 구상하기로는 본 방식을 통해 사용자들에게 여러 노래를 입력받아
user_naver_lyrics_list에 저장된 노래의 모든 데이터셋을 역추적하고,
해당 데이터 셋을 user_google_lyrics_list에 저장된 노래의 데이터셋과 비교하여 
네이버에 저장되지 않은 노래를 찾아내는데 쓰일 수 있지 않을까 생각합니다.
"""

import selenium
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from itertools import repeat
from selenium import webdriver   # selenium 프레임 워크에서 webdriver 가져오기
import requests
from bs4 import BeautifulSoup as bs
import langid
import re
import os
from google.cloud import translate_v2 as translate

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\dltkd\Downloads\lyricstranslate-af60cbfe125b.json"

target = 'ko'

user_naver_artist_list = []
user_naver_lyrics_list = []
user_google_lyrics_list = []
user_input_lyrics_list = []
test = []
test2 = []
only_korean_lyrics = []
return_lyrics = []
Highlight_Lyrics = []

use_google = False
user_input = False

# 사용자에게 노래 제목을 입력받는 함수
def input_title():
    title = (input("System: 제목을 입력하세요: "))
    return title

# 입력받은 노래 제목을 네이버에서 검색하고 가사를 추출하는 함수
def get_lyrics_naver(title):
    user_naver_lyrics_list.clear()
    user_naver_artist_list.clear()
    url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=" + title + "+가사"
    headers = { "User-Agent":"Mozilla/5.0" }
    html = requests.get(url, headers = headers ).text
    soup = bs(html, "html.parser")
    
    type07 = soup.select('div.intro_box > p')
    lyrics = str(type07)
    lyrics=lyrics.replace('<p class="text no_ellipsis type_center _content_text" style="max-height: 50rem; -webkit-line-clamp: 21;">','"').replace('<br>','').replace('</br>','\n').replace('<br/>','\n').replace('</p>','\n').replace('  ','\n')
    user_naver_lyrics_list.append(lyrics)
    
    type08 = soup.select('span.txt > a')
    artist = str(type08)
    front_index = artist.find('>')
    artist=artist.replace(artist[0:front_index+1],' ').replace('</a>]',' ')
    user_naver_artist_list.append(artist)
    
    return user_naver_artist_list, user_naver_lyrics_list

# 입력받은 노래의 가사가 네이버와 구글에서 바로 크롤링 되지 않는 경우 사용자에게 가사를 직접 입력받는 함수
def get_lyrics_user_input():
    global user_input
    user_input_lyrics_list.clear()
    user_input = True
    user_input_lyrics_list.append(input("System: 가사를 직접 입력해 주세요: "))
    
    return user_input

# 입력받은 노래 제목을 구글에서 검색하고 가사를 추출하는 함수
def get_lyrics_google(title):
    user_google_lyrics_list.clear()
    keyword = [title]
    
    for i in keyword :
        url = "https://www.google.com/search?q=" + i + "+가사"
        headers = { "User-Agent":"Mozilla/5.0" }
        html = requests.get(url, headers = headers ).text
        soup = bs(html, "html.parser")
        
        try :
            type06 = soup.find("div", {'class':'hwc'})
            user_google_lyrics_list.append(type06.text.replace("\n",""))
            
        except :
            print("System: 가사를 찾지 못하였습니다.")
            get_lyrics_user_input()
            
    return user_google_lyrics_list

# 가사의 언어를 분석하고 보여주는 함수
def classify_lyrics_country(title):
    country_dic = {"en":"영어권", "ja":"일본", "zh":"중국", "fr":"프랑스", "es":"스페인", "ru":"러시아", "ko":"대한민국"}
    
    global only_korean_lyrics
    global detected
    
    if use_google == False:
        detected = langid.classify(user_naver_lyrics_list[0])
    elif use_google != False and user_input == False:
        detected = langid.classify(user_google_lyrics_list[0])
    elif use_google != False and user_input != False:
        detected = langid.classify(user_input_lyrics_list[0])
        
    if detected[0] != 'ko':
        print("System: 입력하신", title, "은(는)", country_dic.get(detected[0]), " 노래 입니다.")
        doTranslate(classify_lyrics_country)
    else:
        print("System: 입력하신", title, "은(는)", country_dic.get(detected[0]), " 노래 입니다.")
        
        for i in range(0, len(user_naver_lyrics_list[0])):
            
            if use_google == False:
                word = user_naver_lyrics_list[0]
            elif use_google != False and user_input == False:
                word = user_google_lyrics_list[0]
            elif use_google != False and user_input != False:
                word = user_input_lyrics_list[0]
                
        if word.upper() != word.lower():
            print("System:", title, " 의 가사에 영어가 있어 해당 단어 번역을 시작합니다 잠시만 기다려주십시오...\n")
            doTranslate(classify_lyrics_country)
        else:
            if use_google == False:
                only_korean_lyrics = (user_naver_lyrics_list[0])
            elif use_google != False and user_input == False:
                only_korean_lyrics = (user_google_lyrics_list[0])
            elif use_google != False and user_input != False:
                only_korean_lyrics = (user_input_lyrics_list[0])
            print("\nSystem:", title, "의 가사입니다\n")
            doTranslate(classify_lyrics_country)
    return detected

# 입력받은 제목을 네이버 또는 구글로 검색하게끔 get_lyrics_naver()와 get_lyrics_google()을 조건에 맞게 실행하는 함수.
# 네이버와 구글에서 찾지 못하는 경우 직접 가사를 입력하는 get_lyrics_user_input() 함수와
# 입력받은 가사의 언어를 분석하는 classify_lyrics_country() 함수가 한번에 돌아가게끔 하는 함수
# 즉 이 함수 하나에 노래 제목 넣으면 됨.
def find_lyrics_algorithm():
    global use_google
    global user_input
    use_google = False
    user_input = False
    user_title = input_title()
    get_lyrics_naver(user_title)
    if len(user_naver_lyrics_list[0]) > 10:
        print("System: 네이버에서", user_title ,"을(를) 찾았습니다")
    else:
        use_google = True
        print("System: 구글로 검색합니다")
        get_lyrics_google(user_title)
    classify_lyrics_country(user_title)

# 입력받은 가사의 언어에 맞는 언어로 번역하는 함수.
def doTranslate(classify_lyrics_country):
    global Final_Lyrics
    translate_client = translate.Client()
    reg_en = re.compile(r'[a-zA-Z]')
    reg_ja = re.compile(r'[一-龯ぁ-んァ-ンｧ-ﾝﾞﾟ\w一-龥A-Z]')
    reg_ch = re.compile(r'[一-龥A-Z]')
    reg_fr = re.compile(r'[a-zA-Zàâçéèêëîïôûùüÿñæœ]')
    # /^[a-zA-ZÀ-Ÿ-. ]*$/ 이어야 한다는 말이 있음.
    reg_es = re.compile(r'[p{Latin}]')
    reg_ru = re.compile(r'[А-яA-Z]')
    
    lyrics_origin = detected[0]
    
    if lyrics_origin == 'en':
        reg = reg_en
        print("System: 영어 노래 가사의 번역을 시작합니다. 잠시만 기다려 주십시오...\n")
    elif lyrics_origin == 'ja':
        reg = reg_ja
        print("System: 일본 노래 가사의 번역을 시작합니다. 잠시만 기다려 주십시오...\n")
    elif lyrics_origin == 'zh':
        reg = reg_ch
        print("System: 중국 노래 가사의 번역을 시작합니다. 잠시만 기다려 주십시오...\n")
    elif lyrics_origin == 'fr':
        reg = reg_fr
        print("System: 프랑스 노래 가사의 번역을 시작합니다. 잠시만 기다려 주십시오...\n")
    elif lyrics_origin == 'es':
        reg = reg_es
        print("System: 스페인 노래 가사의 번역을 시작합니다. 잠시만 기다려 주십시오...\n")
    elif lyrics_origin == 'ru':
        reg = reg_ru
        print("System: 러시아 노래 가사의 번역을 시작합니다. 잠시만 기다려 주십시오...\n")
    else:
        reg = reg_en
    
    if use_google == False:
        Final_Lyrics = pd.DataFrame(user_naver_lyrics_list[0].split(' '))
    elif use_google != False and user_input == False:
        Final_Lyrics = pd.DataFrame(user_google_lyrics_list[0].split(' '))
    elif use_google != False and user_input != False:
        Final_Lyrics = pd.DataFrame(user_input_lyrics_list[0].split(' '))
    
    for i in range(0, len(Final_Lyrics)):
        if reg.match(Final_Lyrics[0][i]):
            translate_target = Final_Lyrics[0][i]
            output = translate_client.translate(translate_target, target_language=target)
            #print(i)
            translate_index = i
            translate_original = Final_Lyrics[0][translate_index]
            #print(output["translatedText"])
            if output["translatedText"] == "NS":
                output["translatedText"] = "나는"
            translate_target = output["translatedText"]
            Final_Lyrics = Final_Lyrics.replace(translate_original, translate_target)
    Make_Full_Lyrics(Final_Lyrics)
    return Final_Lyrics

# 가사를 깔끔하게 정리하는 함수
def Make_Full_Lyrics(Final_Lyrics):
    global test
    global test2
    global test3
    
    test.clear()
    test2.clear()
    
    for i in range(0, len(Final_Lyrics)):
        test.append(Final_Lyrics[0].iloc[i])
            
    for i in range(0, len(test)):
        test2.append(test[i])
        
    test3 = ' '.join(test2)
    print(test3)
    return test3

In [2]:
selected_words = pd.read_csv('selected_words.csv', encoding='cp949')
selected_words = selected_words.columns
#all_docs[0]

In [3]:
all_words = pd.read_csv('all_words.csv', encoding='cp949')
all_words = all_words.columns
#all_docs[0]

In [4]:
selected_words = selected_words.tolist()

In [5]:
all_words = all_words.tolist()

In [6]:
a_lst = []

for a in all_words :
    try:
        a_lst.append(eval(a))
    except:
        a_lst.append(eval(a[:-2]))
        
a_lst

[(['언니/Noun',
   '동생/Noun',
   '으로/Josa',
   '부르다/Verb',
   '맞다/Verb',
   '일/Noun',
   '인가요/Josa',
   '..??/Punctuation'],
  '공포'),
 (['그냥/Noun',
   '내/Noun',
   '느낌/Noun',
   '일/Modifier',
   '뿐/Noun',
   '겠다/Verb',
   '?/Punctuation'],
  '공포'),
 (['아직/Adverb',
   '너무/Adverb',
   '초기/Noun',
   '라서/Josa',
   '그런/Modifier',
   '거/Noun',
   '죠/Josa',
   '?/Punctuation'],
  '공포'),
 (['유치원/Noun', '버스/Noun', '사고/Noun', '낫다/Verb'], '공포'),
 (['근데/Adverb', '원래/Noun', '이렇다/Adjective', '맞다/Verb'], '공포'),
 (['남자친구/Noun', '가/Josa', '떠나다/Verb', '요/Noun'], '공포'),
 (['이/Determiner',
   '거/Noun',
   '하다/Verb',
   '허리/Noun',
   '가/Josa',
   '아프다/Adjective',
   '있다/Adjective',
   '?/Punctuation',
   ';;/Punctuation'],
  '공포'),
 (['내/Noun',
   '가/Josa',
   '불안하다/Adjective',
   '꾸다/Verb',
   '끄다/Verb',
   '../Punctuation'],
  '공포'),
 (['일주일/Noun',
   '도/Josa',
   '안/Noun',
   '남다/Verb',
   '.../Punctuation',
   'ㅠㅠ/KoreanParticle'],
  '공포'),
 (['약/Noun',
   '은/Josa',
   '최대한/Noun',
   '안/VerbPrefix',
   '

In [7]:
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt
loaded_model = joblib.load('Extraction_Emotion.joblib')
global emotion

okt = Okt()

emotion_number_list = ["행복", "놀람", "분노", "공포", "혐오", "슬픔", "중립"]
emotion_result_list = ["1", "2", "3", "4", "5", "6", "7"]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

all_docs_train = [term_frequency(d) for d, _ in a_lst]
all_docs_test = [c for _, c in a_lst]

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

def predict_target_emotion(target):
    token = tokenize(target)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    print(loaded_model.predict(data))
    emotion = int(loaded_model.predict(data))
    
    end = (emotion_number_list[emotion-1])
    
    return end

In [8]:
find_lyrics_algorithm()

System: 제목을 입력하세요: 하루하루
System: 네이버에서 하루하루 을(를) 찾았습니다
System: 입력하신 하루하루 은(는) 대한민국  노래 입니다.
System: 하루하루  의 가사에 영어가 있어 해당 단어 번역을 시작합니다 잠시만 기다려주십시오...

[" 떠나가 이인칭 대명사 나는 마침내 나는 깨닫다 저것 나는 누틴 없이 너 내가 ~였다 그래서 잘못된 용서하다 나 아 아 아 아 
 
 파도처럼 부숴진 내맘 
 바람처럼 흔들리는 내맘 
 연기처럼 사라진 내 사랑 
 문신처럼 지워지지 않아 
 한숨만 땅이 꺼지라 쉬죠 
 내 가슴 속에 먼지만 쌓이죠 말하다 좋은 안녕 
 
 이인칭 대명사 니가 없이는 단 하루도 못 살 것만 같았던 나 
 생각과는 다르게도 그럭저럭 혼자 잘 살아 
 보고싶다고 불러봐도 넌 아무 대답없잖아 
 헛된 기대 걸어봐도 이젠 소용없잖아 
 
 네 옆에 있는 그 사람이 뭔지 혹시 널 울리진 않는지 
 그대 내가 보이긴 하는지 벌써 싹 다 잊었는지 
 걱정 돼 다가가기 조차 말을 걸 수 조차 없어 애 태우고 
 나 홀로 밤을 지새우죠 수 백번 지워내죠 
 
 돌아보지 말고 떠나가라 또 나를 찾지 말고 살아가라 
 너를 사랑했기에 후회 없기에 좋았던 기억만 가져가라 
 그럭저럭 참아볼만 해 그럭저럭 견뎌낼만 해 
 넌 그럴수록 행복해야 돼 하루하루 무뎌져 가네 
 
 오 소녀 나는 울다 울다 나 나의 모두(말하다 좋은 안녕) 
 
 길을 걷다 너와 나 우리 마주친다 해도 
 못 본 척 하고서 그대로 가던 길 가줘 
 
 자꾸만 옛 생각이 떠오르면 아마도 
 나도 몰래 그댈 찾아 갈지도 몰라 
 
 넌 늘 그 사람과 행복하게 
 넌 늘 내가 다른 맘 안 먹게 
 넌 늘 작은 미련도 안 남게끔 
 잘 지내줘 나 보란 듯이 
 
 넌 늘 저 하늘같이 하얗게 
 뜬 구름과도 같이 새파랗게 
 넌 늘 그래 
 그렇게 웃어줘 아무일 없듯이 
 
 돌아보지 말고 떠나가라 또 나를 찾지 말고 살아가라 
 너를 사랑했기에 후회 없기에 좋았던 기

In [9]:
predict_target_emotion(test3)

[6.]


'슬픔'

In [10]:
a = predict_target_emotion(test3)

[6.]


In [11]:
import csv
with open('After_Data_Pretreatment_Processing(13486).csv', 'r', encoding='UTF-8') as f: 
    reader = csv.reader(f) 
    station_info = list(reader)[1:]

In [12]:
import pandas as pd
col_name = ['index', 'title', 'artist', 'lyrics', 'emotion']
Song_File_df = pd.DataFrame(station_info, columns=col_name)

In [13]:
Song_File_df[Song_File_df["emotion"] == a]

,index,title,artist,lyrics,emotion
10,10,우리들만의 추억,서태지와 아이들,한결 같은 너희들의 정성이 우리에겐 너무 커다란 힘이 되었지 편지속에 수많았던 사...,슬픔
12,12,발해를 꿈꾸며,서태지와 아이들,진정 나에겐 단 한가지 내가 소망하는 게 있어 갈려진 땅의 친구들을 언제쯤 볼 수...,슬픔
16,16,Come Back Home,서태지와 아이들,난 지금 무엇을 찾으려고 애를 쓰는 걸까 난 지금 어디로 쉬지않고 흘러가는가 난 ...,슬픔
30,30,제킬 박사와 하이드,서태지와 아이들,내 마음을 철저하게 속이고 살아온 내 인생은 가슴 깊이 존재했던 불만이 있어 너무...,슬픔
31,31,아이들의 눈으로,서태지와 아이들,나에게 꿈이 있어요 모두를 사랑하지요 지금 아이들의 두 눈 속에서 나는 느끼고 있...,슬픔
...,...,...,...,...,...
13441,13441,Summer Hate (Feat. 비),지코 (ZICO),Good morning 하마터면 달 뜰 때까지 쿨쿨쿨 잠잘 뻔 택배 아저씨 초인...,슬픔
13447,13447,반만,진민호,다 알고 있어 네가 만났던 사람 난 한 번도 이길 수 없었어 성격도 안 맞고 싸운...,슬픔
13469,13469,마음이 말하는 행복 (Happiness) (Feat.이라온),마크툽 (MAKTUB),소란했던 오늘의 하루가 너와 함께 고요해 작은 입술을 열어 우리의 밤을 노래해 넌...,슬픔
13473,13473,거짓말이라도 해서 널 보고싶어,백지영,내가 미안해 이렇게 아플 줄도 모르고 너에게 모질게도 대했던 바보야 후회에 맘...,슬픔


In [14]:
b = Song_File_df[Song_File_df["emotion"] == a].index
c = np.random.randint(low=0, high=len(b), size = 10)

In [15]:
# 분석한 노래의 인덱스 값
# str(b[c][9])

In [16]:
for i in range(0, 10):
    print(Song_File_df[Song_File_df["index"] == str(b[c][i])])

     index     title artist  \
2723  2723  아! 현실 감정     폭스   

                                                 lyrics emotion  
2723   알아 너의 마음을 전부 보여준 것을 나를 잊으라는 마음을 너무 슬픈 꿈 속에 나를...      슬픔  
     index  title artist                                             lyrics  \
2608  2608  사랑이야기    god   지금부터 나만의 아름다운 사랑이야기가 시작될꺼야 s-bop bop bop baby...   

     emotion  
2608      슬픔  
     index     title       artist  \
2231  2231  전화하지 말아요  플라이 투 더 스카이   

                                                 lyrics emotion  
2231   뒤척이고 있는 날 알고 있는 듯 한밤중에 걸려온 부재중 전화 한 통 발신 번호 없...      슬픔  
       index    title artist  \
12153  12153  잘자요 굿나잇   트와이스   

                                                  lyrics emotion  
12153   오늘 하루는 어땠나요 힘든 일은 없었나요 그렇게 하루하루 살아가면 되는 거겠죠 조...      슬픔  
     index       title artist  \
3141  3141  Love Story    쥬얼리   

                                                 lyrics emotion  
3141   Yeah Hey Boy This is love story Im sorry I wa...      슬픔  
    index  

In [17]:
find_lyrics_algorithm()

System: 제목을 입력하세요: 感電
System: 구글로 검색합니다
System: 입력하신 感電 은(는) 일본  노래 입니다.
System: 일본 노래 가사의 번역을 시작합니다. 잠시만 기다려 주십시오...

도망 싶은 밤 왕래 행방은 아직 불명 돌고 돌아 虚しくっ하고 곤란 버렸다 원 멍멍 잃은 생각도 없지만 뭔가 부족한 기분 좀 이상으로 높은 상태 피게 넣은 사륜차 형제여 如何か하자 더 이상 아무것도 생각하지 않도록 은하계의 변두리로 안녕 진실도 도덕도 작동하지 않는 오징어 한 밤에도 우리 손뼉을 쳐 웃는 아무도 모르는 채 단 한 순간의 이 반짝임을 먹고 尽くそ 것이다 둘이서 くたばる까지 행운을 빕니다 우리기도를 아직 가자 아무도 따라 잡을 수없는 정도의 속도로 번개처럼 살고 싶어 그냥 너는 어떻게하고 싶어? 대답은 필요 없어 굴렀다 차창과 정경 동기는 아직 알 수없는 방해스럽고 골치 붙어 헤맨 냥 냥냥 ここいら에서 떨어 뜨린 지갑 누군가보고하지 않았나요? 바보처럼 재수 없네 얼버무주고 하이웨이 스타처럼 친구 다른 정 만화 같은 싸움하자 酒落에 무엇 않을 정도의 녀석을 시험에서 정론과 폭론의 분류조차 할 세라 도시를 벗어나 서로 웃는 목표는 메로 엔딩 그것은 심장을 찰나 흔드는 것을 쫓아 순간에 잃어 버리는 분명 영원히이 어딘가에 있구나과 모레를 찾아 헤매는 것도 나쁘지 않을 것이다 네가 어딘가로 사라진 아침부터 이런 밤이 더 아직 나을 폐에 수련 멀리 사이렌 울려 퍼지 경계 사랑하는 것처럼 싸움 하자구 남는 세나가 내건 폐에 수련 멀리 사이렌 울려 퍼지 경계 사랑하는 것처럼 싸움 하자구 남는 세나가 내건 단 한순간의 이 반짝임을 먹고 尽くそ 것이다 둘이서 くたばる까지 행운을 빕니다 우리기도를 아직 가자 아무도 따라 잡을 수없는 정도의 속도로 그것은 심장을 찰나 흔드는 것을 쫓아 순간에 잃어 버리는 분명 영원히이 어딘가에 있구나과 모레를 찾아 헤매는 것도 나쁘지는 않을 것입니다 번개처럼 살고 싶어 그냥 너는 어떻게하고 싶어? 대답은 필요 없어


In [18]:
predict_target_emotion(test3)

[4.]


'공포'

In [19]:
find_lyrics_algorithm()
# 白月光与朱砂痣
"""
从前的歌谣 都在指尖绕 得不到的美好 总在心间挠 白饭粒无处抛 蚊子血也抹不掉
触不可及刚刚好 日久天长让人恼 那时滚烫的心跳 也曾无处遁逃 像一团烈火燃烧 
烧尽跨不过的桥 时光匆匆地跑火焰化作月遥遥 再无激荡的波涛 也从不在梦里飘摇 
白月光在照耀 你才想起她的好 朱砂痣久难消 你是否能知道 窗前的明月照 你独自一人远眺 
白月光是年少 是她的笑  那时滚烫的心跳 也曾无处遁逃 像一团烈火燃烧 烧尽跨不过的桥 
时光匆匆地跑 火焰化作月遥遥 再无激荡的波涛 也从不在梦里飘摇 白月光在照耀 你才想起她的好 
朱砂痣久难消 你是否能知道 窗前的明月照 你独自一人远眺 白月光是年少 是她的笑 白月光在照耀 
你才想起她的好 朱砂痣久难消 你是否能知道 窗前的明月照 你独自一人远眺 白月光是年少 是她的笑
"""

System: 제목을 입력하세요: 白月光与朱砂痣
System: 구글로 검색합니다
System: 가사를 찾지 못하였습니다.
System: 가사를 직접 입력해 주세요: 从前的歌谣 都在指尖绕 得不到的美好 总在心间挠 白饭粒无处抛 蚊子血也抹不掉 触不可及刚刚好 日久天长让人恼 那时滚烫的心跳 也曾无处遁逃 像一团烈火燃烧  烧尽跨不过的桥 时光匆匆地跑火焰化作月遥遥 再无激荡的波涛 也从不在梦里飘摇  白月光在照耀 你才想起她的好 朱砂痣久难消 你是否能知道 窗前的明月照 你独自一人远眺  白月光是年少 是她的笑  那时滚烫的心跳 也曾无处遁逃 像一团烈火燃烧 烧尽跨不过的桥  时光匆匆地跑 火焰化作月遥遥 再无激荡的波涛 也从不在梦里飘摇 白月光在照耀 你才想起她的好  朱砂痣久难消 你是否能知道 窗前的明月照 你独自一人远眺 白月光是年少 是她的笑 白月光在照耀  你才想起她的好 朱砂痣久难消 你是否能知道 窗前的明月照 你独自一人远眺 白月光是年少 是她的笑
System: 입력하신 白月光与朱砂痣 은(는) 중국  노래 입니다.
System: 중국 노래 가사의 번역을 시작합니다. 잠시만 기다려 주십시오...

올드 발라드 당신의 손끝에서 모든 얻을 수 없는 아름다움 항상 내 마음을 긁어 쌀알을 던질 곳이 없다 모기의 피는 닦을 수 없다 손이 닿지 않는 곳에 오랜만에 짜증난다 그때의 뜨거운 심장 박동 도망칠 곳이 없었다 불처럼 타오르는  건널 수 없는 다리를 불태워 시간은 빠르게 흘러 불꽃은 저 멀리 달이 되어 거센 파도는 이제 그만 꿈에서 절대 흔들리지 마라  하얗게 빛나는 달빛 당신은 그녀를 생각합니다 사마귀 두더지는 오랫동안 제거하기 어렵습니다. 알 수 있니? 창 앞의 밝은 달 당신은 혼자  바이 웨광은 젊다 그녀의 미소입니다  그때의 뜨거운 심장 박동 도망칠 곳이 없었다 불처럼 타오르는 건널 수 없는 다리를 불태워  시간은 빠르게 흘러 불꽃은 멀리 달로 변해 거센 파도는 이제 그만 꿈에서 절대 흔들리지 마라 하얗게 빛나는 달빛 당신은 그녀를 생각합니다  사마귀 두더지는 오랫동안 제거하기 어렵습니다. 알 수 있니? 창 앞의 밝은

'\n从前的歌谣 都在指尖绕 得不到的美好 总在心间挠 白饭粒无处抛 蚊子血也抹不掉\n触不可及刚刚好 日久天长让人恼 那时滚烫的心跳 也曾无处遁逃 像一团烈火燃烧 \n烧尽跨不过的桥 时光匆匆地跑火焰化作月遥遥 再无激荡的波涛 也从不在梦里飘摇 \n白月光在照耀 你才想起她的好 朱砂痣久难消 你是否能知道 窗前的明月照 你独自一人远眺 \n白月光是年少 是她的笑  那时滚烫的心跳 也曾无处遁逃 像一团烈火燃烧 烧尽跨不过的桥 \n时光匆匆地跑 火焰化作月遥遥 再无激荡的波涛 也从不在梦里飘摇 白月光在照耀 你才想起她的好 \n朱砂痣久难消 你是否能知道 窗前的明月照 你独自一人远眺 白月光是年少 是她的笑 白月光在照耀 \n你才想起她的好 朱砂痣久难消 你是否能知道 窗前的明月照 你独自一人远眺 白月光是年少 是她的笑\n'

In [20]:
predict_target_emotion(test3)

[5.]


'혐오'

In [21]:
find_lyrics_algorithm()
# Гимн Сове́тского Сою́за
""" Союз нерушимый республик свободных Сплотила навеки 
Великая Русь. Да здравствует созданный волей народов Единый, 
могучий Советский Союз!  Славься, Отечество наше свободное, 
Дружбы народов надёжный оплот! Знамя советское, знамя народное 
Пусть от победы к победе ведёт!  Сквозь грозы сияло нам солнце свободы,
И Ленин великий нам путь озарил: Нас вырастил Сталин — на верность народу,
На труд и на подвиги нас вдохновил!  Славься, Отечество наше свободное,
Счастья народов надёжный оплот! Знамя советское, знамя народное 
Пусть от победы к победе ведёт!  Мы армию нашу растили в сраженьях. 
Захватчиков подлых с дороги сметём! Мы в битвах решаем судьбу поколений, 
Мы к славе Отчизну свою поведём!  Славься, Отечество наше свободное, 
Славы народов надёжный оплот! Знамя советское, знамя народное 
Пусть от победы к победе ведёт! """

System: 제목을 입력하세요: Гимн Сове́тского Сою́за
System: 구글로 검색합니다
System: 가사를 찾지 못하였습니다.
System: 가사를 직접 입력해 주세요: Союз нерушимый республик свободных Сплотила навеки  Великая Русь. Да здравствует созданный волей народов Единый,  могучий Советский Союз!  Славься, Отечество наше свободное,  Дружбы народов надёжный оплот! Знамя советское, знамя народное  Пусть от победы к победе ведёт!  Сквозь грозы сияло нам солнце свободы, И Ленин великий нам путь озарил: Нас вырастил Сталин — на верность народу, На труд и на подвиги нас вдохновил!  Славься, Отечество наше свободное, Счастья народов надёжный оплот! Знамя советское, знамя народное  Пусть от победы к победе ведёт!  Мы армию нашу растили в сраженьях.  Захватчиков подлых с дороги сметём! Мы в битвах решаем судьбу поколений,  Мы к славе Отчизну свою поведём!  Славься, Отечество наше свободное,  Славы народов надёжный оплот! Знамя советское, знамя народное  Пусть от победы к победе ведёт! 
System: 입력하신 Гимн Сове́тского Сою́за 은(는) 러시아  노래 입니다.
Syste

' Союз нерушимый республик свободных Сплотила навеки \nВеликая Русь. Да здравствует созданный волей народов Единый, \nмогучий Советский Союз!  Славься, Отечество наше свободное, \nДружбы народов надёжный оплот! Знамя советское, знамя народное \nПусть от победы к победе ведёт!  Сквозь грозы сияло нам солнце свободы,\nИ Ленин великий нам путь озарил: Нас вырастил Сталин — на верность народу,\nНа труд и на подвиги нас вдохновил!  Славься, Отечество наше свободное,\nСчастья народов надёжный оплот! Знамя советское, знамя народное \nПусть от победы к победе ведёт!  Мы армию нашу растили в сраженьях. \nЗахватчиков подлых с дороги сметём! Мы в битвах решаем судьбу поколений, \nМы к славе Отчизну свою поведём!  Славься, Отечество наше свободное, \nСлавы народов надёжный оплот! Знамя советское, знамя народное \nПусть от победы к победе ведёт! '

In [22]:
predict_target_emotion(test3)

[2.]


'놀람'

In [23]:
def Extraction_Highlight_Lyrics(test3):
    global return_lyrics
    global Highlight_Lyrics
    
    for i in range(0, len(test3)):
        return_lyrics = (test3.split('\n'))
        
    df = pd.DataFrame(return_lyrics)
    
    for i in range(0, len(df)):
        if df[0][i] == ' ' or df[0][i] == '':
            df = df.drop(i)
            
    list_from_df = []
    list_from_df.append(df.value_counts().to_frame()[df.value_counts().to_frame()[0]>=2].index.tolist())
    Highlight_Lyrics = list_from_df[0]
    
    return Highlight_Lyrics

def Extraction_Highlight_Lyrics_Result(test3):
    Extraction_Highlight_Lyrics(test3)
    if len(Highlight_Lyrics)>=2:
        print("저희가 하이라이트 구간 추출을 할 수 있는 노래입니다. 다음의 가사로 정서 분석을 시작합니다.")
        print(Highlight_Lyrics)
        
    else:
        print("저희가 하이라이트 구간 추출을 할 수 없는 노래입니다. 다음의 가사로 정서 분석을 시작합니다.")
        print(test3)

In [24]:
Extraction_Highlight_Lyrics_Result(test3)

저희가 하이라이트 구간 추출을 할 수 없는 노래입니다. 다음의 가사로 정서 분석을 시작합니다.
노동 조합 파괴할 수 없는 공화국 무료 결합력 있는 영원히  엄청난 러시아. 예 잘 만들어진 의지로 사람들 하나의,  거대한 옛 소련 노동 조합!  영광 받으소서 조국 우리의 무료,  우정 사람들 믿을 수있는 요새! 배너 옛 소련, 배너 사람들  하자 ~에서 승리 나는 승리 리드!  을 통해 뇌우 잉크 우리를 해 자유, 그리고 레닌 큰 우리를 방법 조명: 우리를 높은 스탈린 — ~에 충의 사람들, 에 노동 그리고 ~에 위업 우리를 탁월한!  영광 받으소서 조국 우리의 무료, 행복 사람들 믿을 수있는 요새! 배너 옛 소련, 배너 사람들  하자 ~에서 승리 나는 승리 리드!  우리 군대 우리의 성장 V 전투.  침입자 비겁한 ~와 함께 도로 쓸어버려! 우리 V 전투 우리는 결정한다 운명 세대,  우리 나는 영광 조국 나의 리드하자!  영광 받으소서 조국 우리의 무료,  명성 사람들 믿을 수있는 요새! 배너 옛 소련, 배너 사람들  하자 ~에서 승리 나는 승리 리드! 


In [25]:
Highlight_Lyrics

[]

In [26]:
predict_target_emotion(test3)

[2.]


'놀람'

In [27]:
test_list = []
for i in range(0, len(Highlight_Lyrics)):
    test_list.append(Highlight_Lyrics[i])

In [28]:
test_list

[]

In [29]:
for i in range(0, len(test_list)):
    predict_target_emotion(str(test_list[i]))